In [54]:
# %matplotlib widget

from __future__ import annotations


import matplotlib.colors as mcolors
from test_utilities import test, TestDict

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 10: Factory ---</h2><p>Just across the hall, you find a large factory. Fortunately, the Elves here have plenty of time to decorate. Unfortunately, it's because the factory machines are all offline, and none of the Elves can figure out the initialization procedure.</p>
<p>The Elves do have the manual for the machines, but the section detailing the initialization procedure was eaten by a <a href="https://en.wikipedia.org/wiki/Shiba_Inu">Shiba Inu</a>. All that remains of the manual are some indicator light diagrams, button wiring schematics, and <a href="3">joltage</a> requirements for each machine.</p>
<p>For example:</p>
<pre><code>[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}
[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}
</code></pre>
<p>The manual describes one machine per line. Each line contains a single indicator light diagram in <code>[</code>square brackets<code>]</code>, one or more button wiring schematics in <code>(</code>parentheses<code>)</code>, and joltage requirements in <code>{</code>curly braces<code>}</code>.</p>
<p>To start a machine, its <em>indicator lights</em> must match those shown in the diagram, where <code>.</code> means <em>off</em> and <code>#</code> means <em>on</em>. The machine has the number of indicator lights shown, but its indicator lights are all <em>initially off</em>.</p>
<p>So, an indicator light diagram like <code>[.##.]</code> means that the machine has four indicator lights which are initially off and that the goal is to simultaneously configure the first light to be off, the second light to be on, the third to be on, and the fourth to be off.</p>
<p>You can <em>toggle</em> the state of indicator lights by pushing any of the listed <em>buttons</em>. Each button lists which indicator lights it toggles, where <code>0</code> means the first light, <code>1</code> means the second light, and so on. When you push a button, each listed indicator light either turns on (if it was off) or turns off (if it was on). You have to push each button an integer number of times; there's no such thing as "<span title="But only because these aren't A buttons.">0.5 presses</span>" (nor can you push a button a negative number of times).</p>
<p>So, a button wiring schematic like <code>(0,3,4)</code> means that each time you push that button, the first, fourth, and fifth indicator lights would all toggle between on and off. If the indicator lights were <code>[#.....]</code>, pushing the button would change them to be <code>[...##.]</code> instead.</p>
<p>Because none of the machines are running, the joltage requirements are irrelevant and can be safely ignored.</p>
<p>You can push each button as many times as you like. However, to save on time, you will need to determine the <em>fewest total presses</em> required to correctly configure all indicator lights for all machines in your list.</p>
<p>There are a few ways to correctly configure the first machine:</p>
<pre><code>[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}</code></pre>
<ul>
<li>You could press the first three buttons once each, a total of <code>3</code> button presses.</li>
<li>You could press <code>(1,3)</code> once, <code>(2,3)</code> once, and <code>(0,1)</code> twice, a total of <code>4</code> button presses.</li>
<li>You could press all of the buttons except <code>(1,3)</code> once each, a total of <code>5</code> button presses.</li>
</ul>
<p>However, the fewest button presses required is <code><em>2</em></code>. One way to do this is by pressing the last two buttons (<code>(0,2)</code> and <code>(0,1)</code>) once each.</p>
<p>The second machine can be configured with as few as <code><em>3</em></code> button presses:</p>
<pre><code>[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}</code></pre>
<p>One way to achieve this is by pressing the last three buttons (<code>(0,4)</code>, <code>(0,1,2)</code>, and <code>(1,2,3,4)</code>) once each.</p>
<p>The third machine has a total of six indicator lights that need to be configured correctly:</p>
<pre><code>[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}</code></pre>
<p>The fewest presses required to correctly configure it is <code><em>2</em></code>; one way to do this is by pressing buttons <code>(0,3,4)</code> and <code>(0,1,2,4,5)</code> once each.</p>
<p>So, the fewest button presses required to correctly configure the indicator lights on all of the machines is <code>2</code> + <code>3</code> + <code>2</code> = <code><em>7</em></code>.</p>
<p>Analyze each machine's indicator light diagram and button wiring schematics. <em>What is the fewest button presses required to correctly configure the indicator lights on all of the machines?</em></p>
</article>


In [55]:
from dataclasses import dataclass
from pprint import pprint
import re
from typing import ClassVar, Self

from more_itertools import one
from pyparsing import deque


tests: list[TestDict] = [
    {
        "name": "Example",
        "s": """
            [.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
            [...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}
            [.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}
        """,
        "expected": 7,
    },
]


@dataclass
class Machine:
    diagram: list[bool]
    buttons: list[list[int]]
    requirements: list[int]

    SQUARE_PATTERN: ClassVar[re.Pattern] = re.compile(r"\[(.*?)\]")
    PAREN_PATTERN: ClassVar[re.Pattern] = re.compile(r"\((.*?)\)")
    CURLY_PATTERN: ClassVar[re.Pattern] = re.compile(r"\{(.*?)\}")

    @classmethod
    def from_str(cls, s: str) -> Self:
        return cls(
            diagram=[c == "#" for c in one(cls.SQUARE_PATTERN.findall(s))],
            buttons=[
                [int(i) for i in w.split(",")] for w in cls.PAREN_PATTERN.findall(s)
            ],
            requirements=[int(i) for i in one(cls.CURLY_PATTERN.findall(s)).split(",")],
        )

    def fewest_presses(self) -> int:
        queue = deque([[False] * len(self.diagram)])
        seen = set()
        presses = 0

        while queue:
            for _ in range(len(queue)):
                diagram = queue.popleft()

                if diagram == self.diagram:
                    return presses

                if tuple(diagram) in seen:
                    continue

                seen.add(tuple(diagram))

                for button in self.buttons:
                    queue.append(
                        [b if i not in button else not b for i, b in enumerate(diagram)]
                    )

            presses += 1

        return -1


@test(tests=tests)
def part_I(s: str) -> int:
    return sum(Machine.from_str(l).fewest_presses() for l in s.strip().splitlines())


Test Example passed, for part_I.
Success


In [56]:
with open("../input/day10.txt") as f:
    puzzle = f.read()

print(f"Part I: {part_I(puzzle)}")

Part I: 428


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>428</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>All of the machines are starting to come online! Now, it's time to worry about the joltage requirements.</p>
<p>Each machine needs to be configured to <em>exactly the specified joltage levels</em> to function properly. Below the buttons on each machine is a big lever that you can use to switch the buttons from configuring the indicator lights to increasing the joltage levels. (Ignore the indicator light diagrams.)</p>
<p>The machines each have a set of <em>numeric counters</em> tracking its joltage levels, one counter per joltage requirement. The counters are all <em>initially set to zero</em>.</p>
<p>So, joltage requirements like <code>{3,5,4,7}</code> mean that the machine has four counters which are initially <code>0</code> and that the goal is to simultaneously configure the first counter to be <code>3</code>, the second counter to be <code>5</code>, the third to be <code>4</code>, and the fourth to be <code>7</code>.</p>
<p>The button wiring schematics are still relevant: in this new joltage configuration mode, each button now indicates which counters it affects, where <code>0</code> means the first counter, <code>1</code> means the second counter, and so on. When you push a button, each listed counter is <em>increased by <code>1</code></em>.</p>
<p>So, a button wiring schematic like <code>(1,3)</code> means that each time you push that button, the second and fourth counters would each increase by <code>1</code>. If the current joltage levels were <code>{0,1,2,3}</code>, pushing the button would change them to be <code>{0,2,2,4}</code>.</p>
<p>You can push each button as many times as you like. However, your finger is getting sore from all the button pushing, and so you will need to determine the <em>fewest total presses</em> required to correctly configure each machine's joltage level counters to match the specified joltage requirements.</p>
<p>Consider again the example from before:</p>
<pre><code>[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}
[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}
</code></pre>
<p>Configuring the first machine's counters requires a minimum of <code><em>10</em></code> button presses. One way to do this is by pressing <code>(3)</code> once, <code>(1,3)</code> three times, <code>(2,3)</code> three times, <code>(0,2)</code> once, and <code>(0,1)</code> twice.</p>
<p>Configuring the second machine's counters requires a minimum of <code><em>12</em></code> button presses. One way to do this is by pressing <code>(0,2,3,4)</code> twice, <code>(2,3)</code> five times, and <code>(0,1,2)</code> five times.</p>
<p>Configuring the third machine's counters requires a minimum of <code><em>11</em></code> button presses. One way to do this is by pressing <code>(0,1,2,3,4)</code> five times, <code>(0,1,2,4,5)</code> five times, and <code>(1,2)</code> once.</p>
<p>So, the fewest button presses required to correctly configure the joltage level counters on all of the machines is <code>10</code> + <code>12</code> + <code>11</code> = <code><em>33</em></code>.</p>
<p>Analyze each machine's joltage requirements and button wiring schematics. <em>What is the fewest button presses required to correctly configure the joltage level counters on all of the machines?</em></p>
</article>


In [57]:
from dataclasses import dataclass
from functools import cache
from math import inf
from pprint import pprint
import re
from typing import ClassVar, Self, override

from more_itertools import one
import numpy as np
from pyparsing import deque
from scipy.optimize import milp, LinearConstraint


tests: list[TestDict] = [
    {
        "name": "Example",
        "s": """
            [.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
            [...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}
            [.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}
        """,
        "expected": 33,
    },
]


class MachineII(Machine):
    @override
    def fewest_presses(self) -> int:
        A = np.zeros((len(self.requirements), len(self.buttons)))
        for i in range(len(self.buttons)):
            A[:, i][self.buttons[i]] = 1

        c = np.ones(A.shape[1])
        B = LinearConstraint(A, lb=self.requirements, ub=self.requirements)  # type: ignore
        return sum(milp(c, integrality=c, constraints=B).x)


@test(tests=tests)
def part_II(s: str) -> int:
    return int(
        sum(MachineII.from_str(l).fewest_presses() for l in s.strip().splitlines())
    )


Test Example passed, for part_II.
Success


<link href="style.css" rel="stylesheet"></link>


In [58]:
print(f"Part II: {part_II(puzzle)}")

Part II: 16613


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>16613</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
